# Description

Analyze the clickstream data to identify the most common user journeys leading to TripAdvisor. 

What patterns or sequences of sites or pages do users typically navigate through before reaching TripAdvisor? 
Are there specific categories, themes, or domains that are common in these preTripAdvisor sessions? 

Interpret your findings (visualization and statistical analysis is optional, but it will increase your valuation as a candidate).

# Imports Libraries

In [54]:
# standard libraries
import os
import pandas as pd
from collections import Counter

# external libraries
import plotly.express as px
import tldextract

# Data cleaning

In [2]:
def get_clean_data(file: str) -> pd.DataFrame:
    '''Returns the data cleaned after the process of:
           ordering it, 
           add domain,
           and subdomain. 
       It also saves the new file for future iteration'''
    if os.path.exists(f'data/clean_{file}'):
        return pd.read_parquet(f'data/clean_{file}')
    
    data = order_data(pd.read_parquet(f'data/{file}'))
    data['domain'] = data['targeturl'].apply(get_domain)
    data['sub_domain'] = data['targeturl'].apply(get_sub_domain)
    data.to_parquet(f'data/clean_{file}')
    return data

In [3]:
def order_data(data: pd.DataFrame) -> pd.DataFrame:
    '''Returns the data ordered by the id and the timestamp'''
    sorted_ids = dict(zip(data.userid.unique(), range(len(data.userid.unique()))))
    data['sortedids'] = data.userid.map(sorted_ids)
    data = data.sort_values(['sortedids', 'eventtimestamp'])
    data.drop(columns=['sortedids'])
    return data

In [4]:
def get_domain(url: str) -> str:
    '''Returns the domain of an url,
    using the library tldextract'''
    try:
        return tldextract.extract(url)[1]
    except:
        return np.nan

In [5]:
def get_sub_domain(url: str) -> str:
    '''Returns the sub domain of an url,
    using the library tldextract'''
    try:
        return tldextract.extract(url)[0]
    except:
        return np.nan

# Data Transformation

In [10]:
def get_user_journeys(data: pd.DataFrame, 
                      max_timestamp: int=14400,
                      n_file: int=0) -> pd.DataFrame:
    '''journey refers to a series of clicks, 
    page views, and user actions occurring within a specific 
    time frame that ultimately leads to TripAdvisor.
    This function returns a DataFrame with these user journeys
    to obtain insights into user preferences, 
    behaviors, and decision-making processes.
    The function receives the data as a dataframe and a max 
    timestamp to separate journeys'''

    # start variables id and current journey
    n = 0
    current_id = f'{data.userid[0]}_{n}'

    # base
    users = data.userid.unique()
    user_journey = [tldextract.extract(data.referrerurl.iloc[0])[1]]
    user_journeys = pd.DataFrame(index=range(len(data) // len(users)))

    # check every row
    for idx in range(len(user_journeys.index) - 1):
        row = data.iloc[idx]
        if data.domain.iloc[idx] == 'tripadvisor':
            user_journey.append(data.domain.iloc[idx])
            user_journeys[current_id] = pd.Series(user_journey)

            # start another user journey for new user
            user_journey = []
            n += 1
            current_id = f'{data.userid[idx]}_{n}'
            

        elif data.userid[idx] != current_id[:36]:
            # start another user journey for new user
            user_journey = []
            
            # new id
            n = 0
            current_id = f'{data.userid[idx]}_{n}'

        
        elif (data.eventtimestamp.iloc[idx + 1] \
              - data.eventtimestamp.iloc[idx]) > max_timestamp:
            # start another user journey for new user
            user_journey = []
            current_id = f'{data.userid[idx]}_{n}'
            
        
        # append the url in the same journey
        user_journey.append(data.domain[idx])

    
    # drop empty rows and save the file
    user_journeys = user_journeys.dropna(how='all')
    user_journeys.to_parquet(f'data/user_journeys_{n_file}.parquet')
    
    return user_journeys

# Data Import

In [7]:
# prepare and import all data.
user_journeys = pd.DataFrame()
for n in range(48):
    file = f'data_{n}.parquet'
    data = get_clean_data(file)
    new_user_journeys = get_user_journeys(data, n_file=n)
    user_journeys = pd.concat([user_journeys, new_user_journeys], axis=1)

In [9]:
# save and display user journeys
user_journeys.to_parquet('data/complete_user_journeys.parquet')
user_journeys

,0caeb7a6-1496-4ede-9483-4b685881478f_0,0caeb7a6-1496-4ede-9483-4b685881478f_1,0caeb7a6-1496-4ede-9483-4b685881478f_2,0caeb7a6-1496-4ede-9483-4b685881478f_3,0caeb7a6-1496-4ede-9483-4b685881478f_4,0caeb7a6-1496-4ede-9483-4b685881478f_5,0caeb7a6-1496-4ede-9483-4b685881478f_6,0caeb7a6-1496-4ede-9483-4b685881478f_7,0caeb7a6-1496-4ede-9483-4b685881478f_8,0caeb7a6-1496-4ede-9483-4b685881478f_9,...,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_7,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_8,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_9,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_10,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_11,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_12,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_13,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_14,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_15,4f33a1af-02f1-4205-b38f-54ebbaf88bcf_16
0,wikipedia,google,google,ralphs,overdrive,amazon,amazon,google,doccafe,friendsofcc,...,amazon,amazon,amazon,amazon,amazon,amazon,amazon,tripadvisor,amazon,amazon
1,apple,google,johnnys-shop,amazon,tripadvisor,tripadvisor,tripadvisor,tripadvisor,tripadvisor,tripadvisor,...,amazon,9xbud,tripadvisor,tripadvisor,tripadvisor,tripadvisor,medicare,tripadvisor,tripadvisor,tripadvisor
2,myreadingmanga,google,google,google,NaN,NaN,NaN,NaN,NaN,NaN,...,amazon,xgaytube,NaN,NaN,NaN,NaN,viator,NaN,NaN,NaN
3,arigatomina,arigatomina,id,pressedjuicery,NaN,NaN,NaN,NaN,NaN,NaN,...,americangreetings,amazon,NaN,NaN,NaN,NaN,outbrain,NaN,NaN,NaN
4,mediafire,google,google,google,NaN,NaN,NaN,NaN,NaN,NaN,...,medicare,amazon,NaN,NaN,NaN,NaN,domesticatedcompanion,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Statistical Analysis

## Descriptive Analysis

In [175]:
def get_pages_visited(user_journeys: pd.DataFrame) -> list:
    '''Returns a list pages visited sorted alphabetically'''
    # we have to make the dataframe to a single column
    pages_visited = list(pd.melt(user_journeys).value.dropna())

    # clean and order
    pages_visited = [word for word in pages_visited if word not in ['tripadvisor']]
    pages_visited = sorted(pages_visited)
    return pages_visited

In [60]:
def get_descriptive_data(pages_visited: list):
    '''Returns descriptive statistics for analysis'''
    unique_pages = len(pd.Series(pages_visited).unique())
    
    top_25_pages = dict(Counter(pages_visited).most_common(5)).keys()
    top_25_percentages = dict(Counter(pages_visited).most_common(5)).values()
    
    top_5_pages = dict(Counter(pages_visited).most_common(5)).keys()
    top_5_percentages = dict(Counter(pages_visited).most_common(5)).values()
    
    top_page = top_5_pages[0]
    top_percentage = top_5_pages[1]
    
    te = Counter(pages_visited).most_common(5)
    sum(dict(te).values()) / len(pages_visited)
    users = len(user_journeys.columns)
    return users, unt, unique_pages, most_comon, frequency, per_80

In [ ]:
def get_most_visite_before_tripadvisor(user_journeys: pd.DataFrame) -> dict:
    return most_visited_before_tripadvisor

In [ ]:
# get descriptive statistics
users,  = get_descriptive_data(pages_visited)

In [173]:
pages_visited = get_pages_visited(user_journeys)

In [105]:
f'There was a total 48 files, where {users} where analyzed in {user_journeys} user journeys. \
The most common page visited was {top_page}, with a frequency of {top_percentage}. \
, this is where we should focus, this is where the engagement will come from \
to enhance the pressence in {top_page}, being google the center of the internet.\
A third of the trafic comes only from 5 pages {top_5} \ This is where' 

NameError: name 'users' is not defined

# Visualization 

In [162]:
def plot_most_common_pages(pages_visited: list) -> None:
    '''plot in an histogram the amount of times each page was visited in all data'''
    
    # create the plot
    plot = px.histogram(pages_visited,
                       x=pages_visited,
                       title='Most visited pages')
    
    # order values
    plot.update_xaxes(categoryorder="total descending")
    # Show the histogram
    plot.show()

In [185]:
def plot_most_common_by_category(pages_visited: list) -> None:
    # Create a sample list of values
    top_25 = dict(Counter(pages_visited).most_common(25))
    values = top_25.keys()

    # Create a DataFrame to count the occurrences
    value_counts = pd.DataFrame({'Value': values})
    value_counts['Count'] = list(top_25.values())

    # Group by 'Value' and sum the counts
    #value_counts = value_counts.groupby('Value').sum().reset_index()

    size = list(top_25.values()) 

    # Create a bubble chart using Plotly Express
    color = ['search', 'consumer / finance', 'travel', 'consumer / finance', 
             'social media', 'social media', 'travel', 'consumer / finance', 
             'travel', 'travel', 'search', 'travel', 'travel', 'consumer / finance', 
             'other', 'travel', 'travel', 'travel', 'travel', 'social media',
             'travel', 'search', 'social media', 'other', 'other']

    fig = px.scatter(value_counts,
                     x='Value', 
                     y='Count', 
                     size=size, 
                     title='Value Frequencies by category', 
                     color=color)
    fig.show()

In [176]:
plot_most_common_pages(top_25)

## Conclusion

Google is the number one by far, we should focus on SEO to improve the possibilities for users for their journey to finish in tripadvisor

### Add Categories

In [186]:
plot_most_common_by_category(pages_visited)

### Conclusion

In [ ]:
the other 2 categories most visited during the journey are social media

In [ ]:
# add most common journey to trip advisor here